In [37]:
import os
import sys
import warnings

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import data_loader
import datetime
from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import matplotlib.pyplot as plt
import astropy.time

import importlib
importlib.reload(data_loader)

import numpy as np

%matplotlib qt

In [38]:
start = datetime.datetime(year = 2012, month = 1, day = 1)
end = datetime.datetime(year = 2020, month = 1, day = 1)

OMNI = data_loader.load_raw_data_from_config(
    id=["OMNI", "ONE_HOUR_RESOLUTION"], start=start, end=end
)

UNIX_TIME = cdfepoch.unixtime(OMNI["Epoch"])
KP = OMNI["KP"].astype(np.float64)

invalid_omni_times = (UNIX_TIME < 0) | (KP < 0) | (KP >= 99)
KP[invalid_omni_times] = np.nan

In [39]:
def find_max_in_last_24h(timestamps, values):
    """
    Find the maximum value in the preceding 24 hours for each timestamp.

    Args:
        timestamps: List of Unix timestamps (in seconds)
        values: List of corresponding values

    Returns:
        List of maximum values for each timestamp, considering the previous 24 hours
    """
    if len(timestamps) != len(values):
        raise ValueError("Timestamps and values arrays must have the same length")

    result = np.zeros_like(timestamps)
    result[:] = np.nan
    # 24 hours in seconds
    twenty_four_hours = 24 * 60 * 60

    for i, current_time in enumerate(timestamps):
        # Find all values within the last 24 hours
        within_window = ((current_time - twenty_four_hours) <= timestamps) & (timestamps <= current_time)
        window_values = values[within_window]

        if np.any(np.isnan(window_values)):
            continue

        # If no values in the window, append None or handle as needed
        result[i] = np.max(window_values)

    return result

In [43]:
kp_max = find_max_in_last_24h(UNIX_TIME, KP / 10.0)

t = astropy.time.Time(UNIX_TIME, format='unix', scale='utc')


In [46]:
plt.plot(t.utc.datetime, kp_max)
plt.xlabel("Unix Time (s)")
plt.ylabel("Kp")
plt.title("Max Kp in Preceeding 24 Hrs")

Text(0.5, 1.0, 'Max Kp in Preceeding 24 Hrs')

In [49]:
L_ppi = 5.99 - 0.382 * kp_max

plt.plot(t.utc.datetime, L_ppi)
plt.xlabel("Unix Time (s)")
plt.ylabel("L")
plt.title("Plasmapause Location")

Text(0.5, 1.0, 'Plasmapause Location')